# Manipulando dados no excel

Usaremos 2 bibliotecas para manipulação de dados:
 * [openpyxl](https://openpyxl.readthedocs.io/en/stable/)
 * [pandas](https://pandas.pydata.org/)

### Usaremos agora a **pandas**

In [29]:
import pandas as pd

#### Indicando o arquivo em excel
_Obs:_ É possível usar tanto extensões od tipo ".xlsx" e ".csv"

In [30]:
workSpace = pd.read_excel("models\municipio_mensal.xlsx", sheet_name="Consumo MWh")

#### Podemos ver o tamanho da planilha usando o método "shape()"

In [31]:
print(workSpace.shape)

(4512, 353)


Também podemos retirar as colunas que não nos interessam, como:
 * tipo 
 * cod_muni
 * agência
 * núcleo
 * unidade
 * cod_classe
 * município (Após a filtragem)

Ou podemos usar para filtrar as colunas "município" e as datas usando:

In [ ]:
# # Seleciona as colunas com base nos critérios definidos
# cols_to_keep = ['municipio'] + df.columns[df.columns.str.contains('/')].tolist()

# # Cria um novo DataFrame apenas com as colunas selecionadas
# df_selected = df.loc[:, cols_to_keep]

# # Imprime o novo DataFrame
# print(df_selected)

In [32]:
# Mantendo no DataFrame apenas os dados com município igual a Florianópolis
workSpace = workSpace.loc[workSpace["município"] == "Florianópolis"]
# Removendo as colunas que não serão utilizadas em nossa análise
workSpace = workSpace.drop(columns=['tipo', 'cod_muni', 'agência', 'núcleo', 'unidade', 'cod_classe', 'município'])

Agora possuímos apenas dados referentes ao município de Florianópolis. Porém a distribuição destes dados no DataFrame não está de forma ideal para analisá-los. Utilizaremos a função **melt()** para despivotar a tabela utilizando a coluna classe como referência, assim alterando o formato de **wide** para **long**

In [33]:
#Utilizando a feature classe como ponto de pivotamento, assim tendo o DataFrame corretamente
#construido
df_floripa = workSpace.melt(id_vars=['classe'])

#Renomeando as colunas restantes para refletir o que estamos analisando
df_floripa.rename(columns = {'variable':'Data','value':'MWh'}, inplace = True)

Agora vamos separar os dados de consumo em relação a sua classe, criando um DataFrame para cada uma.

In [68]:
# Separando os dados de consumo pela classe em diferentes DataFrames
df_res = df_floripa.loc[df_floripa['classe']=='Residencial']
df_ind = df_floripa.loc[df_floripa['classe']=='Industrial']
df_com = df_floripa.loc[df_floripa['classe']=='Comercial']
df_rur = df_floripa.loc[df_floripa['classe']=='Rural']
df_pub = df_floripa.loc[df_floripa['classe']=='Poder Público']
df_ilu = df_floripa.loc[df_floripa['classe']=='Iluminação Pública']
df_ser = df_floripa.loc[df_floripa['classe']=='Serviço Público']
df_pro = df_floripa.loc[df_floripa['classe']=='Próprio']

#print(df_res)

#### Filtrando ainda mais
 * Removendo a coluna "classe"
 * Somando todos os valores da coluna "MWh" que possuem as datas da coluna "Data" iguais

Como podemos perceber acima, as linhas de consumo se repetem, mesmo pertencendo a mesma classe. Assim utilizaremos a função utilizando as funções **groupby()** e **sum()**

In [69]:
df_res = df_res.groupby(['Data'], as_index=False).sum()
df_ind = df_ind.groupby(['Data'], as_index=False).sum()
df_com = df_com.groupby(['Data'], as_index=False).sum()
df_rur = df_rur.groupby(['Data'], as_index=False).sum()
df_pub = df_pub.groupby(['Data'], as_index=False).sum()
df_ilu = df_ilu.groupby(['Data'], as_index=False).sum()
df_ser = df_ser.groupby(['Data'], as_index=False).sum()
df_pro = df_pro.groupby(['Data'], as_index=False).sum()
#print(df_com)

#print(df_res["Data"])

#### Renomeando a coluna "Data" para "Datetime"

In [75]:
df_res = df_res.rename(columns={"Data": "Datetime"})
df_ind = df_ind.rename(columns={"Data": "Datetime"})
df_com = df_com.rename(columns={"Data": "Datetime"})
df_rur = df_rur.rename(columns={"Data": "Datetime"})
df_pub = df_pub.rename(columns={"Data": "Datetime"})
df_ilu = df_ilu.rename(columns={"Data": "Datetime"})
df_ser = df_ser.rename(columns={"Data": "Datetime"})
df_pro = df_pro.rename(columns={"Data": "Datetime"})

#### Convertendo para o formato ".csv"

1) Concatenando as colunas "Data" e "MWh"

In [77]:
df_res_concat = pd.concat([df_res["Datetime"], df_res["MWh"]], axis=1)
df_ind_concat = pd.concat([df_ind["Datetime"], df_ind["MWh"]], axis=1)
df_com_concat = pd.concat([df_com["Datetime"], df_com["MWh"]], axis=1)
df_rur_concat = pd.concat([df_rur["Datetime"], df_rur["MWh"]], axis=1)
df_pub_concat = pd.concat([df_pub["Datetime"], df_pub["MWh"]], axis=1)
df_ilu_concat = pd.concat([df_ilu["Datetime"], df_ilu["MWh"]], axis=1)
df_ser_concat = pd.concat([df_ser["Datetime"], df_ser["MWh"]], axis=1)
df_pro_concat = pd.concat([df_pro["Datetime"], df_pro["MWh"]], axis=1)

2) Convertendo no formato ".csv"
3) Salvando os modelos filtrados

In [74]:
# Salvando as alterações
df_res_concat.to_csv("models/After Filters/res_municipio_mensal.csv", index=False)
df_ind_concat.to_csv("models/After Filters/ind_municipio_mensal.csv", index=False)
df_com_concat.to_csv("models/After Filters/com_municipio_mensal.csv", index=False)
df_rur_concat.to_csv("models/After Filters/rur_municipio_mensal.csv", index=False)
df_pub_concat.to_csv("models/After Filters/pub_municipio_mensal.csv", index=False)
df_ilu_concat.to_csv("models/After Filters/ilu_municipio_mensal.csv", index=False)
df_ser_concat.to_csv("models/After Filters/ser_municipio_mensal.csv", index=False)
df_pro_concat.to_csv("models/After Filters/pro_municipio_mensal.csv", index=False)

## Remover a coluna "Classe" e entrar com "data"